In [8]:
import pandas as pd

df = pd.read_csv('../dados/excertos_lematizados.csv', sep = '|')
df.head()

,COD,DESCR_AREA,EXCERTO_LEMATIZADO
0,1400,Responsabilidade,voto : cuidar o auto de tomada de conta especi...
1,1700,Finanças Públicas,voto : cuidar o auto de solicitação de o congr...
2,5700,Responsabilidade,relatório : tratar se de embargo de declaração...
3,284,Direito Processual,voto : XXSECAO em relação a outro processo jud...
4,298,Pessoal,voto : XXSECAO relativo a o ato envolver o sen...


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

textos = df['EXCERTO_LEMATIZADO'].values
vetorizador = TfidfVectorizer(token_pattern=r'\S+')
vetorizador.fit(textos)
print(len(vetorizador.vocabulary_), 'termos distintos')

26825 termos distintos


In [10]:
matriz = vetorizador.transform(textos)
matriz

<13285x26825 sparse matrix of type '<class 'numpy.float64'>'
	with 2975980 stored elements in Compressed Sparse Row format>

In [11]:
def extrair_tfidf(documento):
    indices = []
    excerto = []
    tf_idf = []
    for termo in textos[documento].split():
        if termo in vetorizador.vocabulary_:
            indice = vetorizador.vocabulary_[termo]
            indices.append(indice)
            excerto.append(termo)
            tf_idf.append(matriz[documento,indice])
    return excerto, indices, tf_idf

In [12]:
excerto, indices, tf_idf = extrair_tfidf(11410)
for i in range(0,21):
    print(excerto[i], '\t', indices[i], '\t', tf_idf[i])

voto 	 25793 	 0.00192289886459048
: 	 17 	 0.010069489365013137
em 	 8803 	 0.05452254470679237
exame 	 9889 	 0.002020733623075538
solicitação 	 23168 	 0.014385556781087145
de 	 6675 	 0.21022420523660734
o 	 17212 	 0.3082171614624362
congresso 	 5325 	 0.011767509614608441
nacional 	 16756 	 0.009748899642027182
, 	 13 	 0.17843772010273884
apresentar 	 1705 	 0.00892965967321472
a 	 101 	 0.05135250520772603
este 	 9695 	 0.008237127228090547
corte 	 6053 	 0.0047916065161832
por 	 19033 	 0.025061525183787892
decisão 	 6731 	 0.0018410690608629656
de 	 6675 	 0.21022420523660734
o 	 17212 	 0.3082171614624362
mesa 	 16032 	 0.005836113396507798
diretor 	 7884 	 0.003179492920519787
de 	 6675 	 0.21022420523660734


In [13]:
len(excerto), len(indices), len(tf_idf)

(3421, 3421, 3421)

In [14]:
def extrair_tamanho(documento):
    qtde = 0
    for termo in textos[documento].split():
        if termo in vetorizador.vocabulary_:
            qtde += 1
    return qtde

In [15]:
extrair_tamanho(11410)

3421

In [20]:
for doc in range(0, len(textos)):
    df.loc[doc, 'tamanho'] = extrair_tamanho(doc)
df['tamanho'].describe()

count    13285.000000
mean       695.828754
std        591.364057
min         25.000000
25%        334.000000
50%        530.000000
75%        856.000000
max       6384.000000
Name: tamanho, dtype: float64

In [21]:
import numpy as np

np.percentile(df['tamanho'], 90), np.percentile(df['tamanho'], 95), np.percentile(df['tamanho'], 99)

(1348.2000000000007, 1742.7999999999993, 3036.16)

In [22]:
def filtrar_texto(excerto, tf_idf, maximo):
    derivado = ''
    original = len(tf_idf)
    if maximo >= original:
        minimo = 0
    else:
        minimo = np.percentile(tf_idf,100-100*maximo/original)
    for indice, valor in enumerate(tf_idf):
        if valor > minimo:
            derivado += excerto[indice] + ' '
    return derivado[:-1]

In [24]:
filtrar_texto(excerto, tf_idf, 2000)

'em de o , de o de o de o , em o de o de o de o , o o de o de de o de o de em o de o de o de o . . o , o de de , o de , o , sigilo , em o de o de o de , em o de o em o de o de o . , , o o em o de o , em o de o . o , em , o de o . o , o de o o de o , , o o sigilo , o , o de sigilo o , , , de o de . . o de em o , , , de o o o em o , , o sigilo , o de o de de o . . , , o de o . . de em o , o o o o , em o de o de o , o o de , em o de o o , de , o o o em o o de o o , o , o , o o , em o . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . o , o , o o de o , o o de . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

In [33]:
for doc in range(0, 5):#len(textos)):
    excerto, _ , tf_idf = extrair_tfidf(doc)
    print('======================================================= Texto', doc, '=======================================================')
    derivado = filtrar_texto(excerto, tf_idf, 1000)
    print(derivado, '(' + str(len(derivado.split())) + ')')

======================================================= Texto 0 =======================================================
voto : cuidar o auto de tomada de conta especial instaurar por o gerência executivo de o instituto nacional de o seguro social em juazeiro de o norte , em desfavor de o exemplo servir e de o senhor , em o condição de segurado de o inss , em decorrência de fraude em o concessão de benefício previdenciários . assistir razão a o unidade técnico . o jurisprudência de este corte ser firme em o sentido de que , não ter ser comprovar o participação de o beneficiário em o concessão regular , por meio de o proceder interno de apurar adotar por o autarquia , não ser possível concluir por o existência de conluio entre este e o exemplo servir que autorizar o concessão , dever , pois , ser afastar a responsabilidade de o segurado . poder ser mencionar em esse mesmo linha o recente , , , todo de o plenário . por fim , em o linha de o jurisprudência de este corte , cumprir comunicar

In [36]:
for doc in range(0, len(textos)):
    excerto, _ , tf_idf = extrair_tfidf(doc)
    df.loc[doc, 'filtrado'] = filtrar_texto(excerto, tf_idf, 1000)
    if (doc + 1) % 1000 == 0:
        print('doc', doc+1, '...')

doc 1000 ...
doc 2000 ...
doc 3000 ...
doc 4000 ...
doc 5000 ...
doc 6000 ...
doc 7000 ...
doc 8000 ...
doc 9000 ...
doc 10000 ...
doc 11000 ...
doc 12000 ...
doc 13000 ...


In [37]:
for doc in range(0, len(textos)):
    df.loc[doc, 'tamanho_derivado'] = len(df.loc[doc, 'filtrado'].split())
df[['tamanho', 'tamanho_derivado']].describe()

,tamanho,tamanho_derivado
count,13285.000000,13285.000000
mean,695.828754,575.627023
std,591.364057,286.420705
min,25.000000,0.000000
25%,334.000000,334.000000
50%,530.000000,529.000000
75%,856.000000,847.000000
max,6384.000000,1426.000000


In [40]:
len(df.query('tamanho != tamanho_derivado'))*100/len(df)

18.712834023334587

In [41]:
np.percentile(df['tamanho'], 90), np.percentile(df['tamanho'], 95), np.percentile(df['tamanho'], 99)

(1348.2000000000007, 1742.7999999999993, 3036.16)

In [42]:
np.percentile(df['tamanho_derivado'], 90), np.percentile(df['tamanho_derivado'], 95), np.percentile(df['tamanho_derivado'], 99)

(999.0, 1000.0, 1000.0)

In [49]:
df.groupby('DESCR_AREA').size().sort_values(ascending = False) / 132.85

DESCR_AREA
Pessoal                    25.540083
Licitação                  20.745201
Responsabilidade           17.636432
Direito Processual         13.631916
Contrato Administrativo     7.083177
Convênio                    5.141137
Competência do TCU          4.162589
Gestão Administrativa       2.544223
Finanças Públicas           2.468950
Desestatização              1.046293
dtype: float64

In [50]:
df.query('tamanho != tamanho_derivado').groupby('DESCR_AREA').size().sort_values(ascending = False) / 132.85 / .18712834023334587

DESCR_AREA
Pessoal                    22.526146
Licitação                  21.600965
Responsabilidade           16.411907
Direito Processual         11.946903
Contrato Administrativo     8.165728
Convênio                    5.792438
Finanças Públicas           5.108608
Competência do TCU          4.465004
Gestão Administrativa       2.212389
Desestatização              1.769912
dtype: float64

introduz um viés...

In [43]:
df.head()

,COD,DESCR_AREA,EXCERTO_LEMATIZADO,tamanho,filtrado,tamanho_derivado
0,1400,Responsabilidade,voto : cuidar o auto de tomada de conta especi...,421.0,voto : cuidar o auto de tomada de conta especi...,421.0
1,1700,Finanças Públicas,voto : cuidar o auto de solicitação de o congr...,371.0,voto : cuidar o auto de solicitação de o congr...,371.0
2,5700,Responsabilidade,relatório : tratar se de embargo de declaração...,255.0,relatório : tratar se de embargo de declaração...,255.0
3,284,Direito Processual,voto : XXSECAO em relação a outro processo jud...,220.0,voto : em relação a outro processo judiciais q...,220.0
4,298,Pessoal,voto : XXSECAO relativo a o ato envolver o sen...,1107.0,": a o ato o senhor , rememorar , desde logo , ...",1000.0


In [51]:
df.to_csv('../dados/excertos_filtrados1000.csv', sep = '|', columns = ['COD', 'DESCR_AREA', 'filtrado'], index=False)

In [3]:
import pandas as pd

df = pd.read_csv('../dados/excertos_lematizados.csv', sep = '|')
df.head()

,COD,DESCR_AREA,EXCERTO_LEMATIZADO
0,1400,Responsabilidade,voto : cuidar o auto de tomada de conta especi...
1,1700,Finanças Públicas,voto : cuidar o auto de solicitação de o congr...
2,5700,Responsabilidade,relatório : tratar se de embargo de declaração...
3,284,Direito Processual,voto : XXSECAO em relação a outro processo jud...
4,298,Pessoal,voto : XXSECAO relativo a o ato envolver o sen...


In [4]:
df.loc[11410]

COD                                                               37522
DESCR_AREA                                           Direito Processual
EXCERTO_LEMATIZADO    voto : em exame solicitação de o congresso nac...
Name: 11410, dtype: object

In [5]:
df.loc[11999]

COD                                                               43342
DESCR_AREA                                                      Pessoal
EXCERTO_LEMATIZADO    voto revisor : tratar se de representação ofer...
Name: 11999, dtype: object

In [6]:
df = pd.read_csv('../dados/excertos_filtrados1000.csv', sep = '|')
df.head()

,COD,DESCR_AREA,filtrado
0,1400,Responsabilidade,voto : cuidar o auto de tomada de conta especi...
1,1700,Finanças Públicas,voto : cuidar o auto de solicitação de o congr...
2,5700,Responsabilidade,relatório : tratar se de embargo de declaração...
3,284,Direito Processual,voto : em relação a outro processo judiciais q...
4,298,Pessoal,": a o ato o senhor , rememorar , desde logo , ..."


In [7]:
df.loc[11410]

COD                        37522
DESCR_AREA    Direito Processual
filtrado                     NaN
Name: 11410, dtype: object